In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import os
import warnings
warnings.filterwarnings('ignore')
from pytorch_toolbelt.inference.tiles import ImageSlicer, CudaTileMerger
from pytorch_toolbelt.utils.torch_utils import tensor_from_rgb_image, to_numpy
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
import pandas as pd
import cv2 

In [ ]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

In [73]:
data_dir = '/home/chandanv/Drive/Competitions/Kaggle/SIIM/SIIM_ISIC/data/raw'
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [ ]:
class patient_dataset(Dataset):
    def __init__(self, df, transforms, data_dir):
        self.df = df
        self.transforms = transforms
        self.data_dir = data_dir
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self,idx):
        labels = self.df.loc[idx, 'target']
        image_names = self.df.loc[idx, 'image_name']

        image_paths = os.path.join(self.data_dir, f'{image_names}.jpg')
        image = cv2.imread(image_paths)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        tiler = ImageSlicer(image.shape, tile_size = (512, 512), tile_step= (512, 512))


